<a href="https://colab.research.google.com/github/Emirat17/Bayesian/blob/main/Covid_Gap_Montecarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files

from google.colab import files
uploaded = files.upload()

Saving data_covid.xlsx to data_covid.xlsx


In [6]:
import io
data = pd.read_excel(io.BytesIO(uploaded['data_covid.xlsx']))
data_covid = data

In [3]:
data.head()

,Tanggal,PROBABLE,SUSPEK,POSITIF,SEMBUH,MENINGGAL
0,2020-09-02 00:00:00,42,243,518,4967,630
1,2020-09-03 00:00:00,39,404,519,5077,636
2,2020-09-04 00:00:00,46,441,482,5198,640
3,2020-09-05 00:00:00,51,486,460,5325,644
4,2020-09-06 00:00:00,58,465,483,5402,648


In [4]:
data.tail()

,Tanggal,PROBABLE,SUSPEK,POSITIF,SEMBUH,MENINGGAL
262,2021-05-22 00:00:00,20,59,398,34328,2881
263,2021-05-23 00:00:00,24,56,385,34433,2886
264,2021-05-24 00:00:00,31,70,358,34480,2894
265,2021-05-25 00:00:00,23,95,392,34594,2900
266,2021-05-26 00:00:00,22,83,401,34692,2906


In [7]:
data = data.loc[0:246,]

In [8]:
data.tail()

,Tanggal,PROBABLE,SUSPEK,POSITIF,SEMBUH,MENINGGAL
242,2021-05-02 00:00:00,12,62,313,32829,2773
243,2021-05-03 00:00:00,15,67,286,32891,2776
244,2021-05-04 00:00:00,9,62,292,32969,2783
245,2021-05-05 00:00:00,9,65,313,33085,2792
246,2021-05-06 00:00:00,12,69,309,33159,2799


Let's first use monte carlo simulation for forecasting 

1. Prediksi Jumlah Kasus "POSITIF"

In [9]:
from scipy.stats import norm
log_returns = np.log(1 + data.POSITIF.pct_change())
u = log_returns.mean() #Mean of the logarithmich return
var = log_returns.var() #Variance of the logarithic return
drift = u - (0.5 * var) #drift / trend of the logarithmic return
stdev = log_returns.std() #Standard deviation of the log return


t_intervals = 20 #I just wanted to forecast 250 time points
iterations = 10000 #I wanted to have 10 different forecast

import random
np.random.seed(10)
daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
#daily_returns actually is some kind of a noise. When we multiply this with the t time price, we can obtain t+1 time price

Let's create a variable S0 (inital stock price) equals to the last  closing price .

In [10]:
S0 = data.POSITIF.iloc[-1]
S0

309

In [11]:
#Let us first create en empty matrix such as daily returns
pos_list = np.zeros_like(daily_returns)
pos_list[0] = S0
pos_list

array([[309., 309., 309., ..., 309., 309., 309.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [12]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    pos_list[t] = pos_list[t - 1] * daily_returns[t]
pos_list = pd.DataFrame(pos_list)
pos_list.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,...,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
1,304.749497,339.169314,307.045854,278.641398,297.744849,340.880070,314.541590,320.106182,293.854114,313.574978,302.826882,294.193202,301.602693,331.271005,322.361488,293.320763,304.731631,292.755057,293.129559,324.031391,292.039104,292.614901,290.014027,321.753564,308.935158,277.093853,306.568972,306.064879,294.622039,323.470981,319.578944,324.276897,318.364510,277.295574,300.773533,281.586306,324.783457,291.427303,309.485858,321.965581,...,317.537949,311.530925,309.287717,308.301740,290.506692,345.633485,293.946259,307.656945,299.822753,313.166000,304.911096,298.809213,331.413130,303.421731,300.854545,302.357918,312.617035,316.872562,291.428908,291.078415,298.710256,310.948781,305.887277,319.592941,298.245027,315.722536,315.999662,303.178684,299.199632,331.800892,297.423786,292.871686,314.948023,310.257242,323.782374,304.985579,313.290695,335.880068,351.603715,332.272837
2,316.121760,363.493757,332.088938,302.587226,300.234257,301.576604,316.306110,303.762666,307.891144,304.116723,324.294042,292.787406,275.352283,309.506821,352.620065,310.505900,276.575044,275.557287,294.037148,297.900651,290.412162,291.493077,289.166245,329.441111,266.902848,266.064418,297.570732,295.207739,310.720620,324.627539,340.521261,317.664146,318.301338,268.438203,274.413852,271.765914,297.076564,284.635903,305.574276,344.427487,...,324.112280,291.533631,298.856868,283.742224,303.743975,359.176351,295.377749,313.994643,295.444798,298.264082,304.387291,285.568735,339.907931,281.904541,287.319720,279.468941,304.068203,323.391986,284.830456,287.898298,309.792128,305.441709,305.449523,329.322584,309.961436,332.406594,309.068668,313.196239,309.812834,355.323867,341.787777,306.652010,314.226529,311.158233,312.447599,303.671600,328.529752,345.891683,349.130189,325.668633
3,308.739430,348.492279,327.393250,290.585581,323.011613,282.803541,284.692838,337.570360,313.854063,303.575942,321.779183,280.565754,292.550852,300.786470,335.205527,310.644870,303.491112,263.647025,282.964605,324.648152,289.509433,292.028661,314.393639,316.681585,267.236928,275.560908,295.232346,304.083888,334.259118,329.284760,375.360963,306.312040,338.527401,278.082205,278.211035,269.025911,302.274093,288.100082,303.079191,319.838124,...,322.466080,257.074361,290.068171,283.467445,285.427511,321.568042,306.021977,345.277829,267.509347,291.016490,297.230488,293.345045,319.217909,264.825610,277.941005,274.918245,318.812178,325.397120,288.661282,265.782254,305.680251,301.892350,287.533993,290.892619,303.653551,354.346490,277.337757,283.581475,304.779244,324.087869,360.723959,356.371601,286.321926,323.957592,319.952495,318.041244,371.842998,319.484401,348.119327,335.201051
4,317.071864,365.092920,347.926402,297.615599,332.544755,259.206625,250.650458,355.020026,319.585253,281.116759,313.862784,277.402288,294.518

In [13]:
positif = data.POSITIF
positif = pd.DataFrame(positif)
frames = [positif, pos_list]
monte_carlo_forecast = pd.concat(frames)

In [14]:
#Let us first create en empty matrix such as daily returns
pos_list = np.zeros_like(daily_returns)
pos_list[0] = S0
pos_list

array([[309., 309., 309., ..., 309., 309., 309.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [15]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    pos_list[t] = pos_list[t - 1] * daily_returns[t]

In [16]:
positif_new = np.zeros([t_intervals,1])
positif_new

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [17]:
for t in range(0, t_intervals):
  positif_new[t] = sum(pos_list[t,:])/iterations
positif_new
#positif_new = pd.DataFrame(positif_new)
#positif_new.head()

array([[309.        ],
       [308.31467186],
       [307.68474294],
       [307.27651943],
       [306.5221135 ],
       [305.54511286],
       [304.90510658],
       [304.21776047],
       [303.56780554],
       [302.90258975],
       [302.11379624],
       [301.60700781],
       [300.85534115],
       [300.23747338],
       [299.74695113],
       [299.04199469],
       [298.41801017],
       [298.0088158 ],
       [297.45521577],
       [296.8828666 ]])

In [26]:
real = np.array(data_covid.loc[247:,'POSITIF'])

In [27]:
gap = np.zeros([20,1])
for t in range(0, t_intervals):
  gap[t] = abs(real[t]-positif_new[t])/real[t]*100

In [28]:
real = real.reshape((20,1))
real.shape
real

array([[328],
       [334],
       [364],
       [347],
       [413],
       [395],
       [377],
       [355],
       [349],
       [340],
       [339],
       [374],
       [392],
       [409],
       [395],
       [398],
       [385],
       [358],
       [392],
       [401]])

In [29]:
positif_new = pd.DataFrame(positif_new)
real = pd.DataFrame(real)
gap = pd.DataFrame(gap)

In [30]:
tabel = pd.DataFrame(real)

In [31]:
tabel['prediksi'] = positif_new
tabel['gap'] = gap

In [32]:
tabel

,0,prediksi,gap
0,328,309.000000,5.792683
1,334,308.314672,7.690218
2,364,307.684743,15.471224
3,347,307.276519,11.447689
4,413,306.522114,25.781571
5,395,305.545113,22.646807
6,377,304.905107,19.123314
7,355,304.217760,14.304856
8,349,303.567806,13.017821
9,340,302.902590,10.911003


2. Prediksi Jumlah Kasus "SUSPEK"

In [33]:
from scipy.stats import norm
log_returns = np.log(1 + data.SUSPEK.pct_change())
u = log_returns.mean() #Mean of the logarithmich return
var = log_returns.var() #Variance of the logarithic return
drift = u - (0.5 * var) #drift / trend of the logarithmic return
stdev = log_returns.std() #Standard deviation of the log return


t_intervals = 20 #I just wanted to forecast 250 time points
iterations = 10000 #I wanted to have 50 different forecast

import random
np.random.seed(10)
daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
#daily_returns actually is some kind of a noise. When we multiply this with the t time price, we can obtain t+1 time price

Let's create a variable S0 (inital).

In [34]:
S0 = data.SUSPEK.iloc[-1]
S0

69

In [42]:
#Let us first create en empty matrix such as daily returns
sus_list = np.zeros_like(daily_returns)
sus_list[0] = S0
sus_list

array([[69., 69., 69., ..., 69., 69., 69.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [43]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    sus_list[t] = sus_list[t - 1] * daily_returns[t]
sus_list = pd.DataFrame(sus_list)
sus_list.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,...,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000,69.000000
1,66.502023,84.917815,67.652295,54.197336,63.061652,85.899442,71.484388,74.406199,61.194978,70.983548,65.547485,61.356410,64.943741,80.467833,75.609166,60.941546,66.493117,60.673385,60.850836,76.506877,60.334957,60.607050,59.383468,75.283834,68.606993,53.512170,67.412507,67.159556,61.560912,76.204946,74.126537,76.639360,73.484618,53.601203,64.536601,55.514730,76.913121,60.046604,68.886688,75.397206,...,73.049514,69.930959,68.785980,68.286078,59.614166,88.660290,61.238822,67.960267,64.071512,70.772237,66.582607,63.577806,80.546719,65.841986,64.576316,65.315831,70.489153,72.700308,60.047359,59.882514,63.529718,69.632800,67.070564,74.133954,63.303914,72.098971,72.243621,65.721568,63.767729,80.762166,62.906421,60.728616,71.695569,69.279542,76.372632,66.619768,70.836627,83.048256,92.197611,81.024823
2,71.930034,98.955931,80.500088,65.087574,63.937138,64.592038,72.025893,65.666604,67.723206,65.841581,76.248591,60.371971,52.471672,68.537775,92.323192,69.044218,53.005482,52.560957,60.962258,62.807550,59.258970,59.764025,58.679805,79.041358,48.865747,48.515792,62.648765,61.518092,69.153336,76.427834,85.245805,72.734263,73.067974,49.510263,52.064050,50.923502,62.411352,56.600787,66.564668,87.496146,...,76.150999,59.783021,63.269040,56.195644,65.657374,96.291421,61.599054,70.829157,61.631000,62.982726,65.975473,57.025427,84.895463,55.367680,57.827326,54.280961,65.817587,75.764952,56.689204,58.093681,68.682187,66.498718,66.502604,78.976410,68.767964,80.676098,68.316332,70.418410,68.692674,93.948309,85.971822,67.102185,70.948705,69.376024,70.034485,65.621641,78.542743,88.348159,90.249146,76.988909
3,67.794528,89.403853,77.516757,59.030145,75.167181,55.480788,56.331122,83.131511,70.387461,65.232200,74.513631,54.483004,59.946106,63.871007,81.807120,68.754122,65.190567,47.266719,55.552996,76.039990,58.531939,59.701953,70.664200,71.844436,48.749826,52.288214,61.208685,65.481803,81.280446,78.543625,105.935507,66.583047,83.670889,53.387544,53.444061,49.498544,64.594922,57.883064,64.988617,73.490802,...,74.877492,44.617898,58.790312,55.778769,56.663749,74.401986,66.439101,87.531176,48.863424,59.230302,62.159140,60.318509,73.165659,47.750792,53.325638,52.010057,72.953396,76.441324,58.140957,48.145748,66.269742,64.408718,57.623576,59.172725,65.270302,92.871781,53.061613,55.830040,65.824368,75.740538,96.734334,94.088717,57.070186,75.671005,73.550849,72.551027,103.681083,73.305266,89.185435,81.804625
4,71.670850,98.912671,88.607702,62.017906,79.912030,45.230511,41.891943,92.788735,72.975286,54.442120,70.024557,52.812748,60.553478,71.373996,81.658979,52.206918,74.435405,41.800681,45.687718,68.944598,63.567297,55.385634,72.042266,74.781017,46.069052,39.096783,69.996770,73.770837,85.600254,68.665676,85.825136,64.044950,90.124696,46.704217,53.268311,48.439456,77.180009,58.383383,55.164110,68.910381,...,72.132752,49.207856,53.417104,50.871076,60.347680,75.

In [44]:
suspek = data.SUSPEK
suspek = pd.DataFrame(suspek)
frames = [suspek, sus_list]
monte_carlo_forecast = pd.concat(frames)

In [45]:
#Let us first create en empty matrix such as daily returns
sus_list = np.zeros_like(daily_returns)
sus_list[0] = S0
sus_list

array([[69., 69., 69., ..., 69., 69., 69.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [46]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    sus_list[t] = sus_list[t - 1] * daily_returns[t]

In [47]:
suspek_new = np.zeros([t_intervals,1])
suspek_new

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [48]:
for t in range(0, t_intervals):
  suspek_new[t] = sum(sus_list[t,:])/iterations
#suspek_new = pd.DataFrame(suspek_new)
#suspek_new.head()

Perbandingan Gap antara hasil prediksi dengan data real.

In [51]:
real = np.array(data_covid.loc[247:,'SUSPEK'])
real

array([72, 70, 71, 82, 77, 62, 70, 68, 76, 85, 85, 88, 86, 60, 57, 59, 56,
       70, 95, 83])

In [52]:
gap = np.zeros([20,1])
for t in range(0, t_intervals):
  gap[t] = abs(real[t]-suspek_new[t])/real[t]*100

In [53]:
real = real.reshape((20,1))
real.shape
real

array([[72],
       [70],
       [71],
       [82],
       [77],
       [62],
       [70],
       [68],
       [76],
       [85],
       [85],
       [88],
       [86],
       [60],
       [57],
       [59],
       [56],
       [70],
       [95],
       [83]])

In [54]:
suspek_new = pd.DataFrame(suspek_new)
real = pd.DataFrame(real)
gap = pd.DataFrame(gap)

In [55]:
tabel = pd.DataFrame(real)

In [56]:
tabel['prediksi'] = suspek_new
tabel['gap'] = gap

In [57]:
tabel

,0,prediksi,gap
0,72,69.000000,4.166667
1,70,68.636490,1.947872
2,71,68.278424,3.833206
3,82,68.053630,17.007768
4,77,67.656207,12.134797
5,62,67.136380,8.284484
6,70,66.798353,4.573781
7,68,66.437614,2.297626
8,76,66.092579,13.036080
9,85,65.750694,22.646242


3. Prediksi Jumlah Kasus "PROBABLE"

In [131]:
from scipy.stats import norm
log_returns = np.log(1 + data.PROBABLE.pct_change())
u = log_returns.mean() #Mean of the logarithmich return
var = log_returns.var() #Variance of the logarithic return
drift = u - (0.5 * var) #drift / trend of the logarithmic return
stdev = log_returns.std() #Standard deviation of the log return

t_intervals = 20 #I just wanted to forecast 250 time points
iterations = 10000 #I wanted to have 50 different forecast

import random
np.random.seed(10)
daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
#daily_returns actually is some kind of a noise. When we multiply this with the t time price, we can obtain t+1 time price

Let's create a variable S0 (inital).

In [132]:
S0 = data.PROBABLE.iloc[-1]
S0

12

In [133]:
#Let us first create en empty matrix such as daily returns
prob_list = np.zeros_like(daily_returns)
prob_list[0] = S0
prob_list

array([[12., 12., 12., ..., 12., 12., 12.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [134]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    prob_list[t] = prob_list[t - 1] * daily_returns[t]
prob_list = pd.DataFrame(prob_list)
prob_list.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,...,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
1,11.434693,15.583539,11.685733,8.824698,10.690801,15.812013,12.530206,13.182270,10.291646,12.419136,11.227247,10.326039,11.096452,14.556705,13.452742,10.237701,11.432754,10.180687,10.218408,13.655330,10.108828,10.166594,9.907375,13.379481,11.894955,8.683658,11.633306,11.578055,10.369642,13.587121,13.119558,13.685281,12.975850,8.701959,11.008431,9.097212,13.747216,10.047687,11.956398,13.405001,...,12.878632,12.186387,11.934267,11.824539,9.956141,16.458317,10.300985,11.753140,10.908064,12.372337,11.452243,10.801732,14.574779,11.291165,11.017011,11.177023,12.309700,12.800718,10.047847,10.012928,10.791387,12.120627,11.558630,13.121220,10.742838,12.666784,12.698974,11.265020,10.842611,14.624165,10.657486,10.192424,12.577101,12.042812,13.624994,11.460338,12.386593,15.150347,17.294241,14.684421
2,12.609539,18.885557,14.541410,11.110250,10.862156,11.003242,12.630824,11.235564,11.683021,11.273490,13.575798,10.100925,8.457202,11.861256,17.297093,11.972356,8.566304,8.475430,10.226155,10.619712,9.865688,9.972289,9.743743,14.208530,7.728059,7.658040,10.585724,10.344372,11.996322,13.616225,15.635410,12.788340,12.862688,7.857364,8.374089,8.142460,10.534949,9.308652,11.430504,16.159922,...,13.553798,9.976303,10.718623,9.224355,11.233564,18.243917,10.361615,12.365649,10.368420,10.657235,11.302529,9.397179,15.554081,9.052585,9.564833,8.828173,11.268288,13.466844,9.327070,9.620658,11.892914,11.416165,11.417009,14.193747,11.911727,14.581684,11.812746,12.274909,11.895214,17.683564,15.804233,11.547518,12.392085,12.045264,12.190221,11.225822,14.095120,16.359455,16.806492,13.742934
3,11.680382,16.581533,13.840772,9.802179,13.311668,9.061874,9.238114,15.122365,12.248969,11.124169,13.165270,8.855990,9.995187,10.831035,14.817925,11.890142,11.115179,7.397684,9.076812,13.507710,9.697532,9.943662,12.309987,12.570929,7.692850,8.406660,10.262521,11.178099,14.697221,14.073366,20.555908,11.416691,15.246724,8.631106,8.642679,7.842775,10.986725,9.561593,11.071592,12.936838,...,13.246735,6.876669,9.751773,9.123552,9.307248,13.140295,11.385444,16.142933,7.715558,9.844287,10.464741,10.073884,12.864400,7.493756,8.618434,8.350067,12.817156,13.598056,9.615573,7.572334,11.348704,10.946634,9.507344,9.832170,11.132398,17.400208,8.564432,9.134174,11.252205,13.440382,18.321669,17.689443,9.391869,13.424759,12.950226,12.727700,20.003515,12.895492,16.530250,14.817353
4,12.512958,18.816349,16.369228,10.418387,14.362176,6.985556,6.339135,17.353436,12.802055,8.833795,12.150095,8.500336,10.107836,12.447362,14.760926,8.377043,13.127291,6.321652,7.075097,11.913287,10.749087,9.028113,12.595132,13.204525,7.149962,5.808348,12.143990,12.979049,15.668858,11.852286,15.721005,10.851473,16.724927,7.275026,8.593297,7.618994,13.743234,9.651318,8.982410,11.905800,...,12.615169,7.772370,8.623706,8.106534,10.064353,13.429026,13.348276,14.343862,8.804337,9.108392,9.401787,7.179459,15.540066,7.53

In [135]:
probable = data.PROBABLE
probable = pd.DataFrame(probable)
frames = [probable, prob_list]
monte_carlo_forecast = pd.concat(frames)

In [136]:
#Let us first create en empty matrix such as daily returns
prob_list = np.zeros_like(daily_returns)
prob_list[0] = S0
prob_list

array([[12., 12., 12., ..., 12., 12., 12.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [137]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    prob_list[t] = prob_list[t - 1] * daily_returns[t]

In [138]:
probable_new = np.zeros([t_intervals,1])
probable_new

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [139]:
for t in range(0, t_intervals):
  probable_new[t] = sum(prob_list[t,:])/iterations
#probable_new = pd.DataFrame(probable_new)
#probable_new.head()

Perbandingan Gap antara hasil prediksi dengan data real.

In [140]:
real = np.array(data_covid.loc[247:,'PROBABLE'])
real

array([12, 18, 26, 27, 20, 20, 19, 20, 19, 17, 17, 16, 17, 22, 22, 20, 24,
       31, 23, 22])

In [141]:
gap = np.zeros([20,1])
for t in range(0, t_intervals):
  gap[t] = abs(real[t]-probable_new[t])/real[t]*100

In [142]:
real = real.reshape((20,1))
real.shape
real

array([[12],
       [18],
       [26],
       [27],
       [20],
       [20],
       [19],
       [20],
       [19],
       [17],
       [17],
       [16],
       [17],
       [22],
       [22],
       [20],
       [24],
       [31],
       [23],
       [22]])

In [143]:
probable_new = pd.DataFrame(probable_new)
real = pd.DataFrame(real)
gap = pd.DataFrame(gap)

In [144]:
tabel = pd.DataFrame(real)

In [145]:
tabel['prediksi'] = probable_new
tabel['gap'] = gap

In [146]:
tabel

,0,prediksi,gap
0,12,12.000000,0.000000
1,18,11.937274,33.681810
2,26,11.873179,54.333925
3,27,11.840649,56.145745
4,20,11.769978,41.150111
5,20,11.671419,41.642903
6,19,11.613665,38.875446
7,20,11.550180,42.249101
8,19,11.489802,39.527359
9,17,11.431772,32.754283


3. Prediksi Jumlah Kasus "MENINGGAL"

In [95]:
from scipy.stats import norm
log_returns = np.log(1 + data.MENINGGAL.pct_change())
u = log_returns.mean() #Mean of the logarithmich return
var = log_returns.var() #Variance of the logarithic return
drift = u - (0.5 * var) #drift / trend of the logarithmic return
stdev = log_returns.std() #Standard deviation of the log return

t_intervals = 20 #I just wanted to forecast 250 time points
iterations = 10000 #I wanted to have 10 different forecast

import random
np.random.seed(10)
daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
#daily_returns actually is some kind of a noise. When we multiply this with the t time price, we can obtain t+1 time price

Let's create a variable S0 (inital).

In [96]:
S0 = data.MENINGGAL.iloc[-1]
S0

2799

In [97]:
#Let us first create en empty matrix such as daily returns
dead_list = np.zeros_like(daily_returns)
dead_list[0] = S0
dead_list

array([[2799., 2799., 2799., ..., 2799., 2799., 2799.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [98]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    dead_list[t] = dead_list[t - 1] * daily_returns[t]
dead_list = pd.DataFrame(dead_list)
dead_list['meninggal'] = dead_list[0]
dead_list.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999,meninggal
0,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,...,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000,2799.000000
1,2813.832080,2836.856757,2815.441208,2794.704650,2808.853568,2837.943939,2820.617413,2824.386870,2806.041311,2819.956356,2812.476206,2806.287771,2811.608728,2831.770775,2825.897400,2805.653122,2813.819517,2805.240671,2805.513800,2827.009568,2804.717620,2805.138371,2803.231727,2825.491196,2816.756787,2793.519555,2815.107959,2814.755174,2806.599087,2826.636922,2824.032326,2827.172631,2823.213613,2793.674377,2811.019325,2796.943133,2827.508721,2804.269719,2817.138856,2825.632943,...,2818.552227,2817.001460,2816.316549,2803.594098,2840.938473,2806.108311,2815.867545,2810.341617,2819.676093,2813.945683,2809.616985,2831.863279,2812.896555,2811.076978,2812.144275,2819.299369,2822.203854,2804.270895,2804.013908,2809.546114,2818.150775,2814.630752,2824.041746,2809.212635,2821.422508,2821.611032,2812.724899,2809.896384,2832.115474,2808.622791,2805.325765,2820.894808,2817.672980,2826.844058,2813.998026,2819.761580,2834.752138,2844.646127,2832.422053,2813.832080
2,2839.640186,2869.998015,2850.316251,2830.193075,2828.510983,2829.472087,2839.766264,2831.028853,2833.940853,2831.280014,2845.165071,2823.105924,2809.936963,2835.070650,2863.367600,2835.766555,2810.885301,2810.096229,2824.021814,2826.830662,2821.355195,2822.153542,2820.431567,2848.579107,2803.275443,2802.603838,2826.592122,2824.876441,2835.915847,2845.387790,2855.764379,2840.692943,2841.126481,2804.500264,2809.206500,2807.132868,2826.234369,2817.041835,2832.311136,2858.246072,...,2822.183442,2827.520655,2816.367279,2831.015586,2867.387628,2825.000302,2838.180508,2825.049132,2827.093149,2831.471767,2817.743862,2855.372333,2814.973988,2819.055909,2813.114383,2831.245611,2844.561590,2817.188426,2819.487825,2835.269592,2832.217542,2832.223059,2848.501044,2835.387568,2850.523794,2834.764801,2837.630266,2835.284023,2865.033751,2856.571887,2833.070644,2838.340077,2836.219820,2837.113142,2830.964208,2847.978211,2859.169605,2861.199513,2846.081706,2839.640186
3,2852.562525,2878.995912,2865.335241,2839.428266,2862.396611,2833.564378,2835.001570,2872.022916,2856.134099,2848.900954,2861.563422,2831.850569,2840.886172,2846.898856,2870.485695,2853.899448,2848.840319,2818.468781,2833.687245,2863.498472,2838.626079,2840.499677,2856.507745,2858.085556,2821.373203,2827.971438,2842.860868,2849.263693,2869.867688,2866.592604,2895.326367,2850.848300,2872.642200,2829.933694,2830.033517,2822.807152,2847.968697,2837.571337,2848.545666,2860.245165,...,2813.055160,2839.042920,2834.070419,2835.558060,2861.420384,2850.642618,2876.964889,2821.592130,2839.748707,2844.321588,2841.472765,2859.822392,2819.426326,2829.824237,2827.468619,2859.545412,2864.001032,2837.991903,2820.200664,2850.400072,2847.694627,2837.146340,2839.656637,2848.956412,2882.650

In [99]:
meninggal = data.MENINGGAL
meninggal = pd.DataFrame(meninggal)
frames = [meninggal, dead_list]
monte_carlo_forecast = pd.concat(frames)

In [100]:
#Let us first create en empty matrix such as daily returns
dead_list = np.zeros_like(daily_returns)
dead_list[0] = S0
dead_list

array([[2799., 2799., 2799., ..., 2799., 2799., 2799.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [101]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    dead_list[t] = dead_list[t - 1] * daily_returns[t]

In [102]:
meninggal_new = np.zeros([t_intervals,1])
meninggal_new

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [103]:
for t in range(0, t_intervals):
  meninggal_new[t] = sum(dead_list[t,:])/iterations
#meninggal_new = pd.DataFrame(meninggal_new)
#meninggal_new.head()

Perbandingan Gap antara hasil prediksi dengan data real.

In [106]:
real = np.array(data_covid.loc[247:,'MENINGGAL'])
real

array([2799, 2808, 2815, 2818, 2819, 2822, 2830, 2832, 2840, 2845, 2847,
       2862, 2870, 2875, 2878, 2881, 2886, 2894, 2900, 2906])

In [107]:
gap = np.zeros([20,1])
for t in range(0, t_intervals):
  gap[t] = abs(real[t]-meninggal_new[t])/real[t]*100

In [108]:
real = real.reshape((20,1))
real.shape
real

array([[2799],
       [2808],
       [2815],
       [2818],
       [2819],
       [2822],
       [2830],
       [2832],
       [2840],
       [2845],
       [2847],
       [2862],
       [2870],
       [2875],
       [2878],
       [2881],
       [2886],
       [2894],
       [2900],
       [2906]])

In [109]:
meninggal_new = pd.DataFrame(meninggal_new)
real = pd.DataFrame(real)
gap = pd.DataFrame(gap)

In [110]:
tabel = pd.DataFrame(real)

In [111]:
tabel['prediksi'] = meninggal_new
tabel['gap'] = gap

In [112]:
tabel

,0,prediksi,gap
0,2799,2799.000000,0.000000
1,2808,2815.985610,0.284388
2,2815,2833.135794,0.644256
3,2818,2850.528454,1.154310
4,2819,2867.777862,1.730325
5,2822,2884.986086,2.231966
6,2830,2902.525553,2.562740
7,2832,2920.130324,3.111946
8,2840,2937.873675,3.446256
9,2845,2955.709536,3.891372


4. Prediksi Jumlah Kasus "SEMBUH"

In [113]:
from scipy.stats import norm
log_returns = np.log(1 + data.SEMBUH.pct_change())
u = log_returns.mean() #Mean of the logarithmich return
var = log_returns.var() #Variance of the logarithic return
drift = u - (0.5 * var) #drift / trend of the logarithmic return
stdev = log_returns.std() #Standard deviation of the log return

t_intervals = 20 #I just wanted to forecast 250 time points
iterations = 10000 #I wanted to have 10 different forecast

import random
np.random.seed(10)
daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))
#daily_returns actually is some kind of a noise. When we multiply this with the t time price, we can obtain t+1 time price

Let's create a variable S0 (inital).

In [114]:
S0 = data.SEMBUH.iloc[-1]
S0

33159

In [115]:
#Let us first create en empty matrix such as daily returns
sem_list = np.zeros_like(daily_returns)
sem_list[0] = S0
sem_list

array([[33159., 33159., 33159., ..., 33159., 33159., 33159.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [116]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    sem_list[t] = sem_list[t - 1] * daily_returns[t]
sem_list = pd.DataFrame(sem_list)
sem_list.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,...,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000,33159.000000
1,33384.184170,33717.302377,33407.445556,33107.902902,33312.233735,33733.046229,33482.291906,33536.815993,33271.602736,33472.731552,33364.586062,33275.163198,33352.048417,33643.668029,33558.669821,33265.994962,33384.002572,33260.036866,33263.982343,33574.761921,33252.481364,33258.559116,33231.019248,33552.792746,33426.465577,33090.798653,33402.627907,33397.527962,33279.660691,33569.369902,33531.686939,33577.121407,33519.843464,33093.033079,33343.530260,33140.214827,33581.984675,33246.011667,33431.989726,33554.843560,...,33511.759165,33452.426341,33430.003169,33420.100613,33236.253083,33776.417918,33272.570639,33413.609126,33333.736397,33468.678474,33385.826312,33323.264986,33645.007039,33370.661661,33344.363450,33359.788568,33463.230542,33505.237371,33246.028651,33242.316701,33322.240885,33446.621322,33395.729329,33531.823206,33317.422113,33493.936027,33496.662764,33368.180566,33327.302417,33648.657636,33308.899166,33261.266074,33486.303802,33439.712614,33572.367037,33386.582936,33469.914743,33686.828416,33830.131979,33653.095562
2,33768.719892,34209.074400,33923.464601,33631.894214,33607.542362,33621.455988,33770.546595,33643.995046,33686.162620,33647.631646,33848.784765,33529.314055,33338.854509,33702.525343,34112.809919,33712.604760,33352.563607,33341.156782,33542.567580,33583.219295,33503.982483,33515.533466,33490.619734,33898.276845,33242.584685,33232.881678,33579.766632,33554.935436,33714.767162,33852.013049,34002.481551,33783.973492,33790.255462,33260.281610,33328.295706,33298.324690,33574.588565,33441.586435,33662.562141,34038.485707,...,33847.024024,33515.966087,33593.206730,33431.830343,33643.802955,34171.169382,33556.727973,33747.572394,33557.434660,33587.018656,33650.408115,33451.740365,33996.794404,33411.680834,33470.718952,33384.790981,33647.133513,33840.037658,33443.706645,33476.966981,33705.406732,33661.206862,33661.286757,33897.145054,33707.115477,33926.474088,33698.095630,33739.601210,33705.615750,34136.995561,34014.196046,33673.560463,33749.884078,33719.170087,33732.110111,33643.059063,33889.564938,34051.885985,34081.342850,33862.071626
3,33967.211846,34351.087775,34152.605648,33776.759246,34109.935928,33691.792293,33712.613467,34249.748183,34019.034407,33914.098140,34097.839524,33666.966682,33797.890052,33885.062585,34227.414736,33986.608571,33913.218720,33473.235901,33693.572226,34125.934135,33765.133396,33792.287989,34024.456740,34047.355506,33515.266756,33610.787126,33826.514967,33919.359264,34218.436791,34170.865840,34588.630341,33942.343927,34258.746144,33639.203415,33640.649114,33536.021400,33900.577471,33749.848449,33908.945229,34078.702382,...,34104.586650,33394.91

In [117]:
sembuh = data.SEMBUH
sembuh = pd.DataFrame(sembuh)
frames = [sembuh, sem_list]
monte_carlo_forecast = pd.concat(frames)

In [118]:
#Let us first create en empty matrix such as daily returns
sem_list = np.zeros_like(daily_returns)
sem_list[0] = S0
sem_list

array([[33159., 33159., 33159., ..., 33159., 33159., 33159.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [119]:
# With a simple for loop, we are going to forecast the next 250 days
for t in range(1, t_intervals):
    sem_list[t] = sem_list[t - 1] * daily_returns[t]

In [120]:
sembuh_new = np.zeros([t_intervals,1])
sembuh_new

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [121]:
for t in range(0, t_intervals):
  sembuh_new[t] = sum(sem_list[t,:])/iterations
#sembuh_new = pd.DataFrame(sembuh_new)
#sembuh_new.head()

Perbandingan Gap antara hasil prediksi dengan data real.

In [124]:
real = np.array(data_covid.loc[247:,'SEMBUH'])
real

array([33253, 33317, 33376, 33442, 33522, 33606, 33679, 33730, 33770,
       33820, 33887, 33972, 34068, 34160, 34248, 34328, 34433, 34480,
       34594, 34692])

In [125]:
gap = np.zeros([20,1])
for t in range(0, t_intervals):
  gap[t] = abs(real[t]-sembuh_new[t])/real[t]*100

In [126]:
real = real.reshape((20,1))
real.shape
real

array([[33253],
       [33317],
       [33376],
       [33442],
       [33522],
       [33606],
       [33679],
       [33730],
       [33770],
       [33820],
       [33887],
       [33972],
       [34068],
       [34160],
       [34248],
       [34328],
       [34433],
       [34480],
       [34594],
       [34692]])

In [127]:
sembuh_new = pd.DataFrame(sembuh_new)
real = pd.DataFrame(real)
gap = pd.DataFrame(gap)

In [128]:
tabel = pd.DataFrame(real)

In [129]:
tabel['prediksi'] = sembuh_new
tabel['gap'] = gap

In [130]:
tabel

,0,prediksi,gap
0,33253,33159.000000,0.282681
1,33317,33415.404569,0.295358
2,33376,33674.676185,0.894883
3,33442,33937.966027,1.483063
4,33522,34199.682571,2.021605
5,33606,34461.294488,2.545065
6,33679,34728.235044,3.115398
7,33730,34996.641874,3.755238
8,33770,35267.586843,4.434666
9,33820,35540.405289,5.086946


**I'm intended to make an hybid model with monte carlo simulation, arma model and maybe LSTM.  
So, to be continued...
I'm open to any kind of suggestion:)**